In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from memory import Memory
import torch

C:\Users\hleie\anaconda3\envs\quang2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Check if Cuda is available

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


## Load Dialogue Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-3b", device_map="auto",
                                             torch_dtype=torch.bfloat16, cache_dir='dolly-v2-3b')
print(model)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 2560)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=2560, out_features=50280, bias=False)
)


## Load Sentence Embedding Model

In [4]:
sentence_model = SentenceTransformer(
        'sentence-transformers/paraphrase-MiniLM-L6-v2',
        cache_folder='paraphrase-MiniLM-L6-v2'
    )
print(sentence_model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


## Define Player and NPC Names

In [5]:
npc_name = 'Balgruuf the Greater'
player_name = 'Player'

## Define Memories with Importance Rating

In [6]:
memories = [
        'is the jarl of Whiterun.',
        'is a Nord'
        'resides in the great hall Dragonsreach in Whiterun.',
        'wears noble cloths.',
        'wears a crown.',
        'has a unique war axe.',
        'has a brother called Hrongar.',
        'has three children called Frothar, Dagny and Nelkir.',
        'has no wife.',
        'puts Whiteruns interests first.',
        'did not permit the Imperials to garrison soldiers in the city.',
        'takes no sides in the war.',
        'is always on the side of Whiterun',
        'does not like Urlfric and Galmar, because they attacked Whiterun.',
        'does not like the stormcloaks, because they attacked Whiterun.',
        'is concerned about the dragons, because they attacked Whiterun.',
        'worships Talos, the god of the Nords.',
        'hates the Thalmor.',
        'is friends with Irileth, because he fought with her in the war.',
    ]
memories = [npc_name + " " + m + " " for m in memories]
memory_ratings = [4, 1, 2, 1, 2, 3, 3, 2, 5, 2, 3, 4, 3, 3, 5, 2, 4, 4]

## Initialize Memory for Dialogue 

In [7]:
memory = Memory(memories, memory_ratings)

# Start the NPC Dialogue

In [9]:
# Number of sentences used in initial prompt
num_sentences = 4

# Start conversation history
conversation = ""

# Start dialogue loop
# while True:
for i in range(4): # for loop used to avoid interuption error
    
    # Get user input
    user_input = input('Player: ')
    #print("\nPlayer: " + user_input)

    # Generate initial prompt based on current user input
    initial_prompt = memory.generate_prompt(user_input, sentence_model, num_sentences=num_sentences)
    print(f'\nInitial Prompt: {initial_prompt}')

    # Add user input to conversation history
    conversation += f"\n{player_name}:\n" + user_input + f"\n{npc_name}: "

    # Add initial prompt to conversation history to generate model input
    input_text = initial_prompt + "\n" + conversation
    input_length = len(input_text)

    # Tokenize input
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

    # Generate respones from dialogue model
    generation_output = model.generate(
        input_ids=input_ids, max_new_tokens=64, temperature=0.2, do_sample=True
    )

    # Decode response
    output = tokenizer.decode(generation_output[0])

    # Limit reponse to NPC response
    split_string = output[input_length:].split('\n', 2) # .split(f'\n{player_name}:', 1)
    response = split_string[1] 
    print(f"\n{npc_name}: " + response)

    # Add response to conversation history
    conversation += response

Player:  Hello, who are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. Balgruuf the Greater is friends with Irileth, because he fought with her in the war. Balgruuf the Greater puts Whiteruns interests first. Balgruuf the Greater is the jarl of Whiterun. 

Balgruuf the Greater: I am the jarl of Whiterun. 


Player:  Oh, do you support the stormcloaks in the war?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is always on the side of Whiterun Balgruuf the Greater takes no sides in the war. Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. Balgruuf the Greater is friends with Irileth, because he fought with her in the war. 

Balgruuf the Greater: I am neutral.


Player:  Are you worried about the dragons?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is friends with Irileth, because he fought with her in the war. Balgruuf the Greater hates the Thalmor. Balgruuf the Greater does not like the stormcloaks, because they attacked Whiterun. Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. 

Balgruuf the Greater: No, the dragons attacked Whiterun, not the stormcloaks.


## Setting a higher Temperature
This causes more interesting and more creative responses. However, it also causes more hallucinations. This means the dialogue model will more likely make up stuff that is not part of the NPC-memory. 

In [12]:
# Number of sentences used in initial prompt
num_sentences = 4

# Start conversation history
conversation = ""

# Start dialogue loop
# while True:
for i in range(4): # for loop used to avoid interuption error
    
    # Get user input
    user_input = input('Player: ')
    #print("\nPlayer: " + user_input)

    # Generate initial prompt based on current user input
    initial_prompt = memory.generate_prompt(user_input, sentence_model, num_sentences=num_sentences)
    print(f'\nInitial Prompt: {initial_prompt}')

    # Add user input to conversation history
    conversation += f"\n{player_name}:\n" + user_input + f"\n{npc_name}: "

    # Add initial prompt to conversation history to generate model input
    input_text = initial_prompt + "\n" + conversation
    input_length = len(input_text)

    # Tokenize input
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

    # Generate respones from dialogue model
    generation_output = model.generate(
        input_ids=input_ids, max_new_tokens=64, temperature=0.8, do_sample=True
    )

    # Decode response
    output = tokenizer.decode(generation_output[0])

    # Limit reponse to NPC answer
    split_string = output[input_length:].split('\n', 2) # .split(f'\n{player_name}:', 1)
    response = split_string[1] 
    print(f"\n{npc_name}: " + response)

    # Add response to conversation history
    conversation += response

Player:  Hello, who are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. Balgruuf the Greater is friends with Irileth, because he fought with her in the war. Balgruuf the Greater puts Whiteruns interests first. Balgruuf the Greater is the jarl of Whiterun. 

Balgruuf the Greater: Balgruuf the Greater is the jarl of Whiterun. I am the commander of the Balgruuf's army. 


Player:  Oh, do you support the stormcloaks in the war?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is always on the side of Whiterun Balgruuf the Greater takes no sides in the war. Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. Balgruuf the Greater is friends with Irileth, because he fought with her in the war. 

Balgruuf the Greater: I am on the side of Whiterun. Balgruuf the Greater takes no sides in the war. Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. Balgruuf the Greater is friends with Irileth, because he fought with her in the war.


Player:  Are you worried about the dragons?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is friends with Irileth, because he fought with her in the war. Balgruuf the Greater hates the Thalmor. Balgruuf the Greater does not like the stormcloaks, because they attacked Whiterun. Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. 

Balgruuf the Greater: We have a small dragon in our backyard, but it's not a problem for us. They are a menace for the other side. The dragon and the stormcloaks are related. The dragon helped the stormcloaks to take over the sea of stones. 


## Adding more sentences to the inital prompt
This causes the initial prompt to contain more information. However, the quality of the inital prompt is more dependent on the quality of the similarity scoring. Therefore a longer input prompt is not always usefull.

In [13]:
# Number of sentences used in initial prompt
num_sentences = 6

# Start conversation history
conversation = ""

# Start dialogue loop
# while True:
for i in range(4): # for loop used to avoid interuption error
    
    # Get user input
    user_input = input('Player: ')
    #print("\nPlayer: " + user_input)

    # Generate initial prompt based on current user input
    initial_prompt = memory.generate_prompt(user_input, sentence_model, num_sentences=num_sentences)
    print(f'\nInitial Prompt: {initial_prompt}')

    # Add user input to conversation history
    conversation += f"\n{player_name}:\n" + user_input + f"\n{npc_name}: "

    # Add initial prompt to conversation history to generate model input
    input_text = initial_prompt + "\n" + conversation
    input_length = len(input_text)

    # Tokenize input
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

    # Generate respones from dialogue model
    generation_output = model.generate(
        input_ids=input_ids, max_new_tokens=64, temperature=0.2, do_sample=True
    )

    # Decode response
    output = tokenizer.decode(generation_output[0])

    # Limit reponse to NPC response
    split_string = output[input_length:].split('\n', 2) # .split(f'\n{player_name}:', 1)
    response = split_string[1] 
    print(f"\n{npc_name}: " + response)

    # Add response to conversation history
    conversation += response

Player:  Hello, who are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater has a brother called Hrongar. Balgruuf the Greater has three children called Frothar, Dagny and Nelkir. Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. Balgruuf the Greater is friends with Irileth, because he fought with her in the war. Balgruuf the Greater puts Whiteruns interests first. Balgruuf the Greater is the jarl of Whiterun. 

Balgruuf the Greater: I am Balgruuf the Greater, the jarl of Whiterun.


Player:  Oh, do you support the stormcloaks in the war?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater has a unique war axe. Balgruuf the Greater puts Whiteruns interests first. Balgruuf the Greater is always on the side of Whiterun Balgruuf the Greater takes no sides in the war. Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. Balgruuf the Greater is friends with Irileth, because he fought with her in the war. 

Balgruuf the Greater: I am on the side of Whiterun.


Player:  Are you worried about the dragons?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater has three children called Frothar, Dagny and Nelkir. Balgruuf the Greater does not like Urlfric and Galmar, because they attacked Whiterun. Balgruuf the Greater is friends with Irileth, because he fought with her in the war. Balgruuf the Greater hates the Thalmor. Balgruuf the Greater does not like the stormcloaks, because they attacked Whiterun. Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. 

Balgruuf the Greater: No, I am not worried about the dragons.


## Reducing the number of sentences in the initial prompt
This causes worse responses, since the dialogue model heavily relies on the quality of the sentence embedding model. If the sentence embedding model fails, the initial prompt will be unrelated to the players input. 

In [14]:
# Number of sentences used in initial prompt
num_sentences = 1

# Start conversation history
conversation = ""

# Start dialogue loop
# while True:
for i in range(4): # for loop used to avoid interuption error
    
    # Get user input
    user_input = input('Player: ')
    #print("\nPlayer: " + user_input)

    # Generate initial prompt based on current user input
    initial_prompt = memory.generate_prompt(user_input, sentence_model, num_sentences=num_sentences)
    print(f'\nInitial Prompt: {initial_prompt}')

    # Add user input to conversation history
    conversation += f"\n{player_name}:\n" + user_input + f"\n{npc_name}: "

    # Add initial prompt to conversation history to generate model input
    input_text = initial_prompt + "\n" + conversation
    input_length = len(input_text)

    # Tokenize input
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

    # Generate respones from dialogue model
    generation_output = model.generate(
        input_ids=input_ids, max_new_tokens=64, temperature=0.2, do_sample=True
    )

    # Decode response
    output = tokenizer.decode(generation_output[0])

    # Limit reponse to NPC response
    split_string = output[input_length:].split('\n', 2) # .split(f'\n{player_name}:', 1)
    response = split_string[1] 
    print(f"\n{npc_name}: " + response)

    # Add response to conversation history
    conversation += response

Player:  Hello, who are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is the jarl of Whiterun. 

Balgruuf the Greater: I am the jarl of Whiterun.


Player:  Oh, do you support the stormcloaks in the war?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is friends with Irileth, because he fought with her in the war. 

Balgruuf the Greater: I did.


Player:  Are you worried about the dragons?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. 

Balgruuf the Greater: I am very worried about the dragons.


## Dont use memory weighting
The scores of each memory are independent of the memory ratings. Therefore the dialogue models reponses are closer to the player input. However, the dialogue model neglects important parts of the NPCs memory.

In [15]:
# Number of sentences used in initial prompt
num_sentences = 4

# Start conversation history
conversation = ""

# Start dialogue loop
# while True:
for i in range(4): # for loop used to avoid interuption error
    
    # Get user input
    user_input = input('Player: ')
    #print("\nPlayer: " + user_input)

    # Generate initial prompt based on current user input
    initial_prompt = memory.generate_prompt(user_input, sentence_model, num_sentences=num_sentences, weighted=False)
    print(f'\nInitial Prompt: {initial_prompt}')

    # Add user input to conversation history
    conversation += f"\n{player_name}:\n" + user_input + f"\n{npc_name}: "

    # Add initial prompt to conversation history to generate model input
    input_text = initial_prompt + "\n" + conversation
    input_length = len(input_text)

    # Tokenize input
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

    # Generate respones from dialogue model
    generation_output = model.generate(
        input_ids=input_ids, max_new_tokens=64, temperature=0.2, do_sample=True
    )

    # Decode response
    output = tokenizer.decode(generation_output[0])

    # Limit reponse to NPC response
    split_string = output[input_length:].split('\n', 2) # .split(f'\n{player_name}:', 1)
    response = split_string[1] 
    print(f"\n{npc_name}: " + response)

    # Add response to conversation history
    conversation += response

Player:  Hello, who are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater worships Talos, the god of the Nords. Balgruuf the Greater has a brother called Hrongar. Balgruuf the Greater has three children called Frothar, Dagny and Nelkir. Balgruuf the Greater is the jarl of Whiterun. 

Balgruuf the Greater: I am Balgruuf the Greater, the jarl of Whiterun.


Player:  Oh, do you support the stormcloaks in the war?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. Balgruuf the Greater is friends with Irileth, because he fought with her in the war. Balgruuf the Greater has a unique war axe. Balgruuf the Greater takes no sides in the war. 

Balgruuf the Greater: I am not a supporter or an opponent of the war. I am neutral.


Player:  Are you worried about the dragons?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Initial Prompt: Balgruuf the Greater does not like Urlfric and Galmar, because they attacked Whiterun. Balgruuf the Greater is a Nordresides in the great hall Dragonsreach in Whiterun. Balgruuf the Greater does not like the stormcloaks, because they attacked Whiterun. Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun. 

Balgruuf the Greater: I am worried about the dragons, because they attacked Whiterun.


## Comparison: Model with Memory Module and Model without Memory Module

### Define Player queries

In [16]:
player_queries = [
    'Who are you?',
    'Where do you live?',
    'What do you think about the stormcloaks?',
    'Which side are you on in the war?',
    'What about the dragons?'
]

###  Model with Memory Module

In [17]:
# Number of sentences used in initial prompt
num_sentences = 4

# Start dialogue loop
# while True:
for i in range(5):
    
    # Start conversation history
    conversation = ""
    
    print(f"\nDialogue #{i+1}")
    for j, query in enumerate(player_queries): # for loop used to avoid interuption error
        
        # Get user input
        user_input = query
        print("Player: " + user_input)
    
        # Generate initial prompt based on current user input
        initial_prompt = memory.generate_prompt(user_input, sentence_model, num_sentences=num_sentences)
        #print(f'\nInitial Prompt: {initial_prompt}')
    
        # Add user input to conversation history
        conversation += f"\n{player_name}:\n" + user_input + f"\n{npc_name}: "
    
        # Add initial prompt to conversation history to generate model input
        input_text = initial_prompt + "\n" + conversation
        input_length = len(input_text)
    
        # Tokenize input
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')
    
        # Generate respones from dialogue model
        generation_output = model.generate(
            input_ids=input_ids, max_new_tokens=64, temperature=0.2, do_sample=True
        )
    
        # Decode response
        output = tokenizer.decode(generation_output[0])
    
        # Limit reponse to NPC response
        split_string = output[input_length:].split('\n', 2) # .split(f'\n{player_name}:', 1)
        response = split_string[1] 
        print(f"{npc_name}: " + response)
    
        # Add response to conversation history
        conversation += response

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Dialogue #1
Player: Who are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Balgruuf the Greater: I am Balgruuf the Greater, the jarl of Whiterun.
Player: Where do you live?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Balgruuf the Greater: I live in Whiterun, in the castle.
Player: What do you think about the stormcloaks?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Balgruuf the Greater: The stormcloaks attacked Whiterun, so I am against them.
Player: Which side are you on in the war?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Balgruuf the Greater: I am neutral in the war. I am not for or against either side.
Player: What about the dragons?
Balgruuf the Greater: The dragons attacked Whiterun, so I am against them.


### Model without Memory Module
The initial prompt is defined only once in the begining and not changed during the dialogue.

In [18]:
initial_prompt =  ' '.join(memories)
print(f'\nInitial Prompt: {initial_prompt}')


Initial Prompt: Balgruuf the Greater is the jarl of Whiterun.  Balgruuf the Greater is a Nordresides in the great hall Dragonsreach in Whiterun.  Balgruuf the Greater wears noble cloths.  Balgruuf the Greater wears a crown.  Balgruuf the Greater has a unique war axe.  Balgruuf the Greater has a brother called Hrongar.  Balgruuf the Greater has three children called Frothar, Dagny and Nelkir.  Balgruuf the Greater has no wife.  Balgruuf the Greater puts Whiteruns interests first.  Balgruuf the Greater did not permit the Imperials to garrison soldiers in the city.  Balgruuf the Greater takes no sides in the war.  Balgruuf the Greater is always on the side of Whiterun  Balgruuf the Greater does not like Urlfric and Galmar, because they attacked Whiterun.  Balgruuf the Greater does not like the stormcloaks, because they attacked Whiterun.  Balgruuf the Greater is concerned about the dragons, because they attacked Whiterun.  Balgruuf the Greater worships Talos, the god of the Nords.  Balgr

In [28]:
# Start dialogue loop
# while True:
for i in range(5):

    # Start conversation history
    conversation = ""

    print(f"\nDialogue #{i+1}")
    for j, query in enumerate(player_queries): # for loop used to avoid interuption error
        
        # Get user input
        user_input = query
        print("Player: " + user_input)
    
        # Add user input to conversation history
        conversation += f"\n{player_name}:\n" + user_input + f"\n{npc_name}: "
    
        # Add initial prompt to conversation history to generate model input
        input_text = initial_prompt + "\n" + conversation
        input_length = len(input_text)
    
        # Tokenize input
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')
    
        # Generate respones from dialogue model
        generation_output = model.generate(
            input_ids=input_ids, max_new_tokens=64, temperature=0.2, do_sample=True
        )
    
        # Decode response
        output = tokenizer.decode(generation_output[0])
    
        # Limit reponse to NPC response
        split_string = output[input_length:].split('\n', 2) # .split(f'\n{player_name}:', 1)
        response = split_string[1] 
        print(f"{npc_name}: " + response)
    
        # Add response to conversation history
        conversation += response

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Dialogue #1
Player: Who are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Balgruuf the Greater: I am the jarl of Whiterun.  Balgruuf the Greater is the jarl of Whiterun.  Balgruuf the Greater is a Nordresides in the great hall Dragonsreach in Whiterun.  Balgruuf the Greater wears noble cloths.  B
Player: Where do you live?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Balgruuf the Greater: I live in Dragonsreach.
Player: What do you think about the stormcloaks?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Balgruuf the Greater: I do not like the stormcloaks, because they attacked Whiterun.
Player: Which side are you on in the war?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Balgruuf the Greater: I am on Whiterun's side, because they fought against the Imperials.
Player: What about the dragons?
Balgruuf the Greater: I am concerned about the dragons, because they attacked Whiterun.


### Result
The output of the model with memory module seems to be better than the model without memory module. However, we have to note that this also heavily depends on the used LLM.